In [52]:
import os
import pandas as pd

# Configure width of display for pandas data frame
pd.set_option('display.width', 400)
# Configure maximum columns shown
pd.set_option('display.max_columns', 15)
# Do not limit number of rows displayed, allows scroll
pd.set_option("display.max_rows", None)

# Create a folder called Data at the root of this project
base_path = '../Data'
column_converters = {'Date': pd.to_datetime,
                     'Type': str,
                     'Company': str,
                     'Cost': float,
                     'Venmo +': float,
                     'Venmo -': float,
                     'Pay': float,
                     'Net': float
                     }

## Specifying Column Data Types
The following are ways to set the specific data types.

### Setting explicitly
```python
for column, dtype in column_converters.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype)
```

### Setting by inference
```python
# Infer data types
df_cleaned = df_cleaned.infer_objects(copy=False)
```

In [53]:
# Main Data frame, empty by default
data = pd.DataFrame()

for root, dirs, files in os.walk(base_path):
    for file in files:
        if 'Excel' in root and '_Expenses.xlsx' in file:
            file_path = os.path.join(root, file)
            # Read in the Excel file with specified sheet of the file
            df_raw = pd.read_excel(file_path, sheet_name='Expenses', converters=column_converters, usecols=lambda col: col in column_converters.keys() )
            df_cleaned = df_raw.dropna(subset=['Type', 'Company'])
            
            # Infer data types
            df_cleaned = df_cleaned.infer_objects(copy=False)
            # Replace all empty cells with 0
            df_cleaned.fillna(0, inplace=True)
            
            data = pd.concat([data, df_cleaned], ignore_index=True)
            print(file_path)

../Data\Excel\Expenses\2021_Expenses.xlsx
../Data\Excel\Expenses\2022_Expenses.xlsx


In [54]:
print(data)

           Date            Type                                            Company       Cost   Venmo +   Venmo -       Pay        Net
0    2021-01-01            Rent                                 One Theater Square      0.000     0.000  1135.000      0.00   1135.000
1    2021-01-01            Rent                                          Utilities      0.000     0.000    46.540      0.00     46.540
2    2021-01-01  Transportation                                            EZ Pass     25.000     0.000     0.000      0.00     25.000
3    2021-01-02         Grocery                                        Whole Foods      0.000     0.000    22.650      0.00     22.650
4    2021-01-04             Pay                                           Tutoring      0.000     0.000     0.000    190.00    190.000
5    2021-01-07   Entertainment                                            Netflix     13.990     0.000     0.000      0.00     13.990
6    2021-01-08      Restaurant                        

In [55]:
# Creating Pivot Table, where row is categories and column is by month. Cell is summed value of that category for the month
pivot_df = pd.DataFrame(data)
pivot_df['Date'] = pd.to_datetime(pivot_df['Date'])  # Ensure Date column is in datetime format

# Adding Year and Month columns
pivot_df['Year'] = pivot_df['Date'].dt.year
pivot_df['Month'] = pivot_df['Date'].dt.to_period('M')

# Group by Type, Year, and Month and sum the Net column
pivot_df = pivot_df.groupby(['Type', 'Year', 'Month'])['Net'].sum().reset_index()
print(pivot_df)

print() 

# Create Pivot Table
pivot_table = pivot_df.pivot_table(index='Type', columns=['Year', 'Month'], values='Net', fill_value=0)
print(pivot_table)

               Type  Year    Month        Net
0          Business  2022  2022-06    159.830
1              Cash  2021  2021-08    200.000
2              Cash  2022  2022-05     50.000
3              Cash  2022  2022-06    235.960
4              Cash  2022  2022-12     50.000
5           Charity  2021  2021-06    100.000
6           Charity  2021  2021-12    200.000
7           Charity  2022  2022-04    246.240
8           Charity  2022  2022-07     25.140
9           Charity  2022  2022-12     13.550
10         Clothing  2021  2021-06     44.450
11         Clothing  2021  2021-10     10.990
12         Clothing  2022  2022-03    345.850
13         Clothing  2022  2022-04   -169.870
14         Clothing  2022  2022-09      4.000
15         Clothing  2022  2022-10     40.000
16         Clothing  2022  2022-12     27.010
17        Education  2021  2021-01     29.990
18        Education  2021  2021-11    129.000
19        Education  2022  2022-11    129.000
20    Entertainment  2021  2021-01

In [56]:
import pandas as pd

# Sample DataFrame creation
data = {
    'Date': ['2021-01-15', '2021-01-20', '2021-02-10', '2021-03-05', '2022-01-22', '2022-02-11'],
    'Type': ['Food', 'Transport', 'Food', 'Transport', 'Food', 'Transport'],
    'Company': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Cost': [10, 20, 15, 10, 20, 30],
    'Venmo +': [5, 5, 7, 5, 10, 15],
    'Venmo -': [2, 3, 4, 1, 2, 4],
    'Pay': [10, 20, 15, 10, 20, 30],
    'Net': [3, 12, 8, 9, 8, 11],
}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])  # Ensure Date column is in datetime format

# Step 1: Create a new column for month and year
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.to_period('M')

# Step 2: Group by Type, Year, and Month and sum the Net column
pivot_df = df.groupby(['Type', 'Year', 'Month'])['Net'].sum().reset_index()

# Step 3: Pivot the DataFrame
pivot_table = pivot_df.pivot_table(index='Type', columns=['Year', 'Month'], values='Net', fill_value=0)

# Step 4: Create separate DataFrames for each year if needed
for year in df['Year'].unique():
    year_df = pivot_table.xs(year, level=0, axis=1)  # Extract data for the specific year
    print(f"\nPivot Table for {year}:\n", year_df)

# Display the final pivot table
print("\nFull Pivot Table:\n", pivot_table)


Pivot Table for 2021:
 Month      2021-01  2021-02  2021-03
Type                                
Food           3.0      8.0      0.0
Transport     12.0      0.0      9.0

Pivot Table for 2022:
 Month      2022-01  2022-02
Type                       
Food           8.0      0.0
Transport      0.0     11.0

Full Pivot Table:
 Year         2021                    2022        
Month     2021-01 2021-02 2021-03 2022-01 2022-02
Type                                             
Food          3.0     8.0     0.0     8.0     0.0
Transport    12.0     0.0     9.0     0.0    11.0
